# Actividad 5.2 (Valores Nulos)
- Miranda Eugenia Colorado Arróniz A01737027

In [18]:
import pandas as pd

# Leer todas las hojas del excel
excel_path = 'Gastos y costos 20-23.xlsx'
sheets = pd.read_excel(excel_path, sheet_name=None)

In [19]:
# Ver los valores nulos por cada columna en cada hoja
for name, df in sheets.items():
    print(f'Hoja: {name}')
    print(df.isnull().sum())
    print('-'*40)

Hoja: 2020
CALOR Y CONTROL S.A DE C.V       3
Unnamed: 1                     193
Unnamed: 2                       4
Unnamed: 3                       4
Unnamed: 4                       4
Unnamed: 5                       4
Unnamed: 6                    2506
Unnamed: 7                       4
Unnamed: 8                     395
Unnamed: 9                      38
Unnamed: 10                    272
Unnamed: 11                      4
Unnamed: 12                      4
GASTO                            1
10418041.6                       1
Unnamed: 15                      5
Unnamed: 16                      4
Unnamed: 17                   3325
Unnamed: 18                      4
dtype: int64
----------------------------------------
Hoja: 2021
CALOR Y CONTROL S.A DE C.V       1
Unnamed: 1                     149
Unnamed: 2                       2
Unnamed: 3                       2
Unnamed: 4                       2
Unnamed: 5                       2
Unnamed: 6                       2
Unnamed: 7    

- Si la variable es de tipo monto o cantidad y tiene valores atípicos, se usará la mediana.
- Si la variable es de tipo temporal, se usará forward fill (ffill).
- Si la variable es de tipo categórico, se usará un string específico ('Sin dato').
- Si la variable es de tipo monto y tiene secuencia temporal, se usará backward fill (bfill).
- Si la variable tiene pocos nulos y no es crítica, se eliminarán las filas.
- Si la variable es de totales o promedios, se usará la media.

In [20]:
# Cambios personalizado por hoja y variable
sheets_sust = {}
cambios = {}

for name, df in sheets.items():
    df_sust = df.copy()
    cambios[name] = {}
    
    for col in df_sust.columns:
        col_str = str(col)
        if 'fecha' in col_str.lower() or 'periodo' in col_str.lower():
            # Forward fill
            df_sust[col] = df_sust[col].fillna(method='ffill')
            cambios[name][col] = 'Forward fill'
            
        elif df_sust[col].dtype in ['float64', 'int64'] and df_sust[col].isnull().sum() > 0:
            if hasattr(df_sust[col], 'skew') and (df_sust[col].skew() > 1 or df_sust[col].skew() < -1):
                # Mediana
                df_sust[col] = df_sust[col].fillna(df_sust[col].median())
                cambios[name][col] = 'Mediana'
                
            elif hasattr(df_sust[col], 'is_monotonic') and (df_sust[col].is_monotonic or df_sust[col].is_monotonic_decreasing):
                # Backward fill
                df_sust[col] = df_sust[col].fillna(method='bfill')
                cambios[name][col] = 'Backward fill'
                
            else:
                # Media
                df_sust[col] = df_sust[col].fillna(df_sust[col].mean())
                cambios[name][col] = 'Media'
                
        elif df_sust[col].dtype == 'object':
            # String específico
            df_sust[col] = df_sust[col].fillna('Sin dato')
            cambios[name][col] = 'Sin dato'
            
        else:
            # Si hay pocos nulos y no es importante, eliminar filas
            df_sust = df_sust.dropna(subset=[col])
            cambios[name][col] = 'Eliminación de filas'
            
    sheets_sust[name] = df_sust

cambios

{'2020': {'CALOR Y CONTROL S.A DE C.V': 'Sin dato',
  'Unnamed: 1': 'Sin dato',
  'Unnamed: 2': 'Sin dato',
  'Unnamed: 3': 'Sin dato',
  'Unnamed: 4': 'Sin dato',
  'Unnamed: 5': 'Sin dato',
  'Unnamed: 6': 'Sin dato',
  'Unnamed: 7': 'Sin dato',
  'Unnamed: 8': 'Sin dato',
  'Unnamed: 9': 'Sin dato',
  'Unnamed: 10': 'Sin dato',
  'Unnamed: 11': 'Sin dato',
  'Unnamed: 12': 'Sin dato',
  'GASTO': 'Sin dato',
  10418041.6: 'Sin dato',
  'Unnamed: 15': 'Sin dato',
  'Unnamed: 16': 'Sin dato',
  'Unnamed: 17': 'Sin dato',
  'Unnamed: 18': 'Sin dato'},
 '2021': {'CALOR Y CONTROL S.A DE C.V': 'Sin dato',
  'Unnamed: 1': 'Sin dato',
  'Unnamed: 2': 'Sin dato',
  'Unnamed: 3': 'Sin dato',
  'Unnamed: 4': 'Sin dato',
  'Unnamed: 5': 'Sin dato',
  'Unnamed: 6': 'Sin dato',
  'Unnamed: 7': 'Sin dato',
  'Unnamed: 8': 'Sin dato',
  'Unnamed: 9': 'Sin dato',
  'Unnamed: 10': 'Sin dato',
  'Unnamed: 11': 'Sin dato',
  'Unnamed: 12': 'Sin dato',
  'SUMA GTO': 'Sin dato',
  12942388.82: 'Sin dato',

- **Variables temporales (fechas, periodos):** Se usó forward fill para mantener la continuidad temporal.
- **Variables numéricas con sesgo fuerte:** Se usó la mediana para evitar el efecto de valores atípicos.
- **Variables numéricas con secuencia temporal:** Se usó backward fill para mantener la coherencia en series.
- **Variables numéricas de totales o promedios:** Se usó la media para mantener la tendencia general.
- **Variables categóricas o de texto:** Se usó el string 'Sin dato' para identificar los registros sin información.
- **Variables con pocos nulos y no críticas:** Se eliminaron las filas para no afectar el análisis.

In [21]:
# Guardar en CSV
with pd.ExcelWriter('Gastos_y_costos_20-23_sin_nulos.xlsx') as writer:
    for name, df in sheets_sust.items():
        df.to_excel(writer, sheet_name=name, index=False)
        df.to_csv(f'{name}_sin_nulos.csv', index=False)